In [1]:
import os
import sys
from pathlib import Path

print("CWD:", os.getcwd())

PROJECT_DIR = Path(os.getcwd()).parent
sys.path.insert(0, str(PROJECT_DIR))

print("PROJECT_DIR:", PROJECT_DIR)
print("src exists:", (PROJECT_DIR / "src").exists())


CWD: c:\Users\Abhishek Karyagol\OneDrive\Tài liệu\infosys\AI_Cricket_Player_Performance_Prediction\notebooks
PROJECT_DIR: c:\Users\Abhishek Karyagol\OneDrive\Tài liệu\infosys\AI_Cricket_Player_Performance_Prediction
src exists: True


In [2]:
import pandas as pd

from src.config import PROCESSED_DIR
from src.feature_engineering import run_feature_engineering


In [3]:
dataset = run_feature_engineering()
dataset.head()


Saved: C:\Users\Abhishek Karyagol\OneDrive\Tài liệu\infosys\AI_Cricket_Player_Performance_Prediction\data\processed\batsman_match_features.csv | Shape: (16515, 14)


,match_id,batter,runs,balls_faced,date,season,venue,team1,team2,winner,runs_last_5_avg,runs_last_10_avg,career_runs_avg,venue_runs_avg
0,548346,A Ashish Reddy,10,10,2012-04-29,2012,Wankhede Stadium,Mumbai Indians,Deccan Chargers,Mumbai Indians,0.00,0.00,0.00,0.0
1,548352,A Ashish Reddy,3,3,2012-05-04,2012,"MA Chidambaram Stadium, Chepauk",Chennai Super Kings,Deccan Chargers,Chennai Super Kings,10.00,10.00,10.00,0.0
2,548359,A Ashish Reddy,8,8,2012-05-08,2012,"Rajiv Gandhi International Stadium, Uppal",Deccan Chargers,Kings XI Punjab,Kings XI Punjab,6.50,6.50,6.50,0.0
3,548373,A Ashish Reddy,10,4,2012-05-18,2012,"Rajiv Gandhi International Stadium, Uppal",Deccan Chargers,Rajasthan Royals,Deccan Chargers,7.00,7.00,7.00,8.0
4,548376,A Ashish Reddy,4,5,2012-05-20,2012,"Rajiv Gandhi International Stadium, Uppal",Deccan Chargers,Royal Challengers Bangalore,Deccan Chargers,7.75,7.75,7.75,9.0


In [4]:
print("Shape:", dataset.shape)
dataset.info()


Shape: (16515, 14)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16515 entries, 0 to 16514
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   match_id          16515 non-null  int64         
 1   batter            16515 non-null  object        
 2   runs              16515 non-null  int64         
 3   balls_faced       16515 non-null  int64         
 4   date              16515 non-null  datetime64[ns]
 5   season            16515 non-null  object        
 6   venue             16515 non-null  object        
 7   team1             16515 non-null  object        
 8   team2             16515 non-null  object        
 9   winner            16475 non-null  object        
 10  runs_last_5_avg   16515 non-null  float64       
 11  runs_last_10_avg  16515 non-null  float64       
 12  career_runs_avg   16515 non-null  float64       
 13  venue_runs_avg    16515 non-null  float64       
dtypes: 

In [5]:
dataset.isnull().sum().sort_values(ascending=False).head(15)


winner              40
match_id             0
runs                 0
balls_faced          0
date                 0
batter               0
season               0
venue                0
team1                0
team2                0
runs_last_5_avg      0
runs_last_10_avg     0
career_runs_avg      0
venue_runs_avg       0
dtype: int64

In [6]:
dataset.describe().T


,count,mean,min,25%,50%,75%,max,std
match_id,16515.0,907862.594308,335982.0,548331.0,980971.0,1254071.0,1426312.0,369668.604422
runs,16515.0,19.98571,0.0,4.0,13.0,29.0,175.0,21.360857
balls_faced,16515.0,15.798971,1.0,5.0,11.0,23.0,77.0,13.934583
date,16515,2016-07-13 12:29:15.204359424,2008-04-18 00:00:00,2012-04-21 00:00:00,2016-05-07 00:00:00,2021-04-21 00:00:00,2024-05-26 00:00:00,NaN
runs_last_5_avg,16515.0,19.683006,0.0,9.666667,18.2,27.8,158.0,13.118083
runs_last_10_avg,16515.0,19.69968,0.0,11.0,19.5,27.3,158.0,11.7588
career_runs_avg,16515.0,19.356292,0.0,12.571429,20.641256,26.615385,158.0,10.294709
venue_runs_avg,16515.0,13.57904,0.0,0.0,7.0,23.732143,158.0,16.723585


In [7]:
dataset.columns


Index(['match_id', 'batter', 'runs', 'balls_faced', 'date', 'season', 'venue',
       'team1', 'team2', 'winner', 'runs_last_5_avg', 'runs_last_10_avg',
       'career_runs_avg', 'venue_runs_avg'],
      dtype='object')

In [8]:
sample_batter = dataset["batter"].value_counts().index[0]
dataset[dataset["batter"] == sample_batter][
    ["date", "match_id", "batter", "runs", "runs_last_5_avg", "runs_last_10_avg", "career_runs_avg", "venue", "venue_runs_avg"]
].head(15)


,date,match_id,batter,runs,runs_last_5_avg,runs_last_10_avg,career_runs_avg,venue,venue_runs_avg
11421,2008-04-20,335986,RG Sharma,0,0.0,0.000000,0.000000,Eden Gardens,0.000000
11422,2008-04-22,335988,RG Sharma,66,0.0,0.000000,0.000000,"Rajiv Gandhi International Stadium, Uppal",0.000000
11423,2008-04-24,335990,RG Sharma,36,33.0,33.000000,33.000000,"Rajiv Gandhi International Stadium, Uppal",66.000000
11424,2008-05-01,335999,RG Sharma,76,34.0,34.000000,34.000000,"Rajiv Gandhi International Stadium, Uppal",51.000000
11425,2008-05-03,336034,RG Sharma,57,44.5,44.500000,44.500000,M Chinnaswamy Stadium,0.000000
11426,2008-05-06,336007,RG Sharma,23,47.0,47.000000,47.000000,"MA Chidambaram Stadium, Chepauk",0.000000
11427,2008-05-09,336011,RG Sharma,5,51.6,43.000000,43.000000,Sawai Mansingh Stadium,0.000000
11428,2008-05-11,336014,RG Sharma,33,39.4,37.571429,37.571429,"Rajiv Gandhi International Stadium, Uppal",59.333333
11429,2008-05-15,336020,RG Sharma,35,38.8,37.000000,37.000000,Feroz Shah Kotla,0.000000
11430,2008-05-18,336024,RG Sharma,6,30.6,36.777778,36.777778,"Rajiv Gandhi International Stadium, Uppal",52.750000


In [ ]:
out_path = PROCESSED_DIR / "batsman_match_features.csv"
print("Saved at:", out_path)
